# Project Description# 

This project is the final project for the "How to win a data science competition" Coursera course.

This project is provided with a challenging time-series dataset consisting of historical daily sales data. The task is to predict total sales for every product sold in every store in the next month. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

# Project Pipeline

* load data
* remove outliers and handle abnormal values
* handle data leakage to avoid over-generalization
* engineer new features, such as revenue, lagged features, rolling stats, et cetera
* explore data trend and visualize
* re-structure data to fit into models
* prepare train/validation/test datasets
* set model performance evaluators
* explore different ways to forecast timeseries data:
*     1. AR/MA models
*     2. LSTM model
*     3. Ensemble method (with Linear regression/XGBoost/Random Forest models)

# **Data extract and preprocessing**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pylab

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 10)

In [ ]:
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test_sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')
sample_sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

**Handle outliers and abnormal values**

In [ ]:
# check if there are outliers
sns.boxplot(x='item_price',data=sales_train)
sales_train = sales_train[sales_train['item_price']<10000]

In [ ]:
sns.boxplot(x='item_cnt_day',data=sales_train)
sales_train = sales_train[sales_train['item_cnt_day']<1000]

In [ ]:
# it looks there are negative numbers in column item_cnt_day, let's take a look
negative_cnt = sales_train[sales_train['item_cnt_day']<0].sort_values(by='item_cnt_day',ascending=True)
g = sns.countplot(x=negative_cnt['item_cnt_day'],palette='viridis',order=negative_cnt["item_cnt_day"].value_counts().index)#[negative_cnt['item_cnt_day']<-1])
g.set(yscale='log')
plt.title('Negative Daily Item Count Distribution')
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2.,
            height + 3,
            '{:.0f}'.format(height),
            ha="center")

There are more than 7k data entries with negative daily item count. In the project this data field is described as "number of products sold". Does a negative number indicate a return&refund?
It's not clear what's the meaning of negative numbers in item_cnt_day yet. For now let's treat it as outliers and remove it from the dataset.

In [ ]:
sales_train = sales_train[sales_train['item_cnt_day']>0]

In [ ]:
# let's check if there are negative values in the item_price column
print(sales_train[sales_train['item_price']<0])
# we can see there is one item with negative price. Let's fill it with median price of all the other items with the same item_id from the same shop on the same day (or in the same month if there is no other items sold from the same day)
m = sales_train[(sales_train['shop_id']==32)&(sales_train['item_id']==2973)&(sales_train['date_block_num']==4)&(sales_train['item_price']>0)]['item_price'].median()
sales_train.loc[sales_train['item_price']<0,'item_price'] = m

In [ ]:
# now sales_train data is in good shape, let's look at the data from the other files. I don't understand Russian, but there are a few name fields that show similar strings, let's manually handle these given the small data size
dup_shop = shops.loc[shops['shop_id'].isin([0,1,10,11,57,58])]
print(dup_shop['shop_name']) #so we can see 0/57, 1/58, 10/11 are dups

sales_train.loc[sales_train['shop_id']==0,'shop_id']=57
test_sub.loc[test_sub['shop_id']==0,'shop_id']=57
sales_train.loc[sales_train['shop_id']==1,'shop_id']=58
test_sub.loc[test_sub['shop_id']==1,'shop_id']=58
sales_train.loc[sales_train['shop_id']==10,'shop_id']=11
test_sub.loc[test_sub['shop_id']==10,'shop_id']=11

In [ ]:
# item_cat data looks fine. Let's look at items data.
items_40 = items[items['item_category_id']==40]
items_40.head()

Most of items data looks fine, though it looks some of the items name contain special character, we don't know if the special characters are legit part of item names, or errors. 
As I won't do anyting with text features (also I don't understand the Russian text in this case), for now I will leave it as is.

In [ ]:
# now that individual tables are handled, let's merge them together. since we don't need the descriptive features (shop name, item name, etc), we only need to join table items to sales_train, so that sales_train can have item_category_id info
sales_train = sales_train.join(items,on='item_id',rsuffix='_').drop(['item_id_','item_name'],axis=1)
sales_train.head()

**Data leakage**

We want to eliminate the issue of over-generalization, as there are some shops/items that appear in trainset but didn't appear in testset. We want the model to only focus on shops/items that are included in the testdata.


In [ ]:
test_shop_ids = test_sub['shop_id'].unique()
test_item_ids = test_sub['item_id'].unique()
# Only shops that exist in test set.
sales_train_lk = sales_train[sales_train['shop_id'].isin(test_shop_ids)]
# Only items that exist in test set.
sales_train_lk = sales_train_lk[sales_train_lk['item_id'].isin(test_item_ids)]
print('train set size before leaking:',sales_train.shape)
print('train set size after leaking:',sales_train_lk.shape)

# **Feature generation**

In [ ]:
# let's convert the date column to the right format
sales_train_lk['date'] = sales_train_lk['date'].apply(lambda x: datetime.datetime.strptime(x,'%d.%m.%Y'))

In [ ]:
# add a revenue feature
sales_train_lk['revenue'] = sales_train_lk['item_price']*sales_train_lk['item_cnt_day']

In [ ]:
monthly_train_lk = sales_train_lk.groupby(['date_block_num','shop_id','item_category_id','item_id'],as_index=False)#['date','item_cnt_day','item_price'].agg({'date':['min','max'],'item_cnt_day':'sum','item_price':'mean'}).reset_index()
monthly_train_lk = monthly_train_lk.agg({'item_price':['sum','mean'],
                                         'item_cnt_day':['sum','mean','count'],
                                         'revenue':['sum','mean']})
monthly_train_lk.head()

In [ ]:
# rename features
monthly_train_lk.columns = ['date_block_num','shop_id','item_category_id','item_id','total_item_price','avg_item_price','total_item_cnt','avg_item_cnt','total_txn_cnt','total_revenue','avg_revenue']

In [ ]:
monthly_train_lk.shape

There are about 6k records with total_item_cnt>20, about 1% of the total data size, I will treat these as outliers and remove these records from the dataset.

In [ ]:
monthly_train_lk = monthly_train_lk[monthly_train_lk['total_item_cnt']<=20]

In [ ]:
monthly_train_lk.shape

Create placeholder for the missing records of combination of date_block_num, shop_id and item_id

In [ ]:
# build a data set with all the possible combinations of ['date_block_num','shop_id','item_id'] so we won't have missing shop_id or item_id that exist in testdata but not in traindata.
train_shop_ids = monthly_train_lk['shop_id'].unique()
train_item_ids = monthly_train_lk['item_id'].unique()
temp_df = []
for i in range(len(monthly_train_lk['date_block_num'].unique())):
    for shop in train_shop_ids:
        for item in train_item_ids:
            temp_df.append([i,shop,item])
temp_df = pd.DataFrame(temp_df,columns=['date_block_num','shop_id','item_id'])
print(temp_df.shape)

In [ ]:
# merge the placeholder set with the train dataset, and fill missing recodes with 0
monthly_train_lk = pd.merge(temp_df,monthly_train_lk,on=['date_block_num','shop_id','item_id'],how='left')
monthly_train_lk.fillna(0,inplace=True)
print(monthly_train_lk.shape)

In [ ]:
# let's generate two more columns based on date_block_num: month and year
monthly_train_lk['month'] = monthly_train_lk['date_block_num'].apply(lambda x: ((x%12)+1))
monthly_train_lk['year'] = monthly_train_lk['date_block_num'].apply(lambda x: ((x//12)+2013))

In [ ]:
# unitary item price
# monthly_train_lk.columns = ['date_block_num','shop_id','item_category_id','item_id','total_item_price','avg_item_price','total_item_cnt','avg_item_cnt','total_txn_cnt','total_revenue','avg_revenue']
monthly_train_lk['unit_item_price'] = monthly_train_lk['total_revenue']//monthly_train_lk['total_item_cnt']
monthly_train_lk['unit_item_price'].fillna(0,inplace=True)
monthly_train_lk[np.isinf(monthly_train_lk)] = 0

In [ ]:
# min and max of unitary item price over the whole sales period
minmax_price = monthly_train_lk.groupby(['item_id'],as_index=False).agg({'unit_item_price':[np.min,np.max]})
minmax_price.columns = ['item_id','min_unit_item_price','max_unit_item_price']
monthly_train_lk = pd.merge(monthly_train_lk,minmax_price,on='item_id',how='left')

In [ ]:
# unit price change compared to min and max unitary price
monthly_train_lk['price_change+'] = monthly_train_lk['unit_item_price'] - monthly_train_lk['min_unit_item_price']
monthly_train_lk['price_change-'] = monthly_train_lk['max_unit_item_price'] - monthly_train_lk['unit_item_price']

In [ ]:
# lagged features
lags = [1,2,3,6,12]
for lag in lags:
    col = ('total_item_cnt_lag_%s' % lag)
    monthly_train_lk[col] = monthly_train_lk.sort_values('date_block_num').groupby(['shop_id','item_category_id','item_id'])['total_item_cnt'].shift(lag)
    monthly_train_lk[col].fillna(0,inplace=True)

In [ ]:
# item sale trend based on lagged features
monthly_train_lk['trend'] = monthly_train_lk['total_item_cnt']
for lag in lags:
    col = ('total_item_cnt_lag_%s' % lag)
    monthly_train_lk['trend'] -= monthly_train_lk[col]
monthly_train_lk['trend'] /= (len(lags)+1)

In [ ]:
# rolling window based features off total_item_cnt
# define features
cnt_min = lambda x: x.rolling(window=3,min_periods=1).min()
cnt_max = lambda x: x.rolling(window=3,min_periods=1).max()
cnt_avg = lambda x: x.rolling(window=3,min_periods=1).mean()
cnt_std = lambda x: x.rolling(window=3,min_periods=1).std()
func_list = [cnt_min,cnt_max,cnt_avg,cnt_std]
func_name = ['rolling_min','rolling_max','rolling_avg','rolling_std']

for i in range(len(func_list)):
    monthly_train_lk[('total_item_cnt_%s'%func_name[i])] = monthly_train_lk.sort_values('date_block_num').groupby(['shop_id','item_category_id','item_id'])['total_item_cnt'].apply(func_list[i])

monthly_train_lk['total_item_cnt_rolling_std'].fillna(0,inplace=True)

In [ ]:
# create the final prediction feature 'item_cnt_next_month'
monthly_train_lk['item_cnt_next_month'] = monthly_train_lk.sort_values('date_block_num').groupby(['shop_id','item_id'])['total_item_cnt'].shift(-1)

In [ ]:
# take a look at the data with all features generated
monthly_train_lk.head().T

# Exploratory data analysis

In [ ]:
monthly_train_lk.describe().T

In [ ]:
# let's first look at overall monthly sales data
month_sum = monthly_train_lk.groupby(['date_block_num','year','month'],as_index=False)['total_item_cnt'].sum()
month_sum_pivot = month_sum.pivot(values='total_item_cnt',columns='year',index='month')

In [ ]:
plt.figure(figsize=(12,5))
sns.lineplot(x='date_block_num',y='total_item_cnt',data=month_sum,label='raw data')
plt.plot(month_sum['total_item_cnt'].rolling(window=3,center=False).mean(),label='rolling mean')
plt.plot(month_sum['total_item_cnt'].rolling(window=3,center=False).std(),label='rolling std')
plt.title('Trend of Items Sold Monthly')
plt.legend()

It looks there is a year-over-year upward trend with obvious spike by the end of each year. Let's take a closer look by breaking down data into each years and compare YOY.

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot()
colors = [plt.cm.viridis(i/float(len(month_sum_pivot.columns))) for i in range(len(month_sum_pivot.columns))]
x = month_sum_pivot.index
for col,color in zip(month_sum_pivot.columns,colors):
    y = month_sum_pivot[col]
    ax.plot(x,y,label=col,c=color)
    x_annotate = x[0]
    y_annotate = month_sum_pivot.iloc[0][col]
    ax.text(x_annotate-0.4,y_annotate,col,fontsize=8,c=color)
ax.set_xlabel('Months', fontsize = 13)
ax.set_ylabel('Items Sold Monthly', fontsize = 13)
ax.set_title('Monthly Sales by Year (2013-2015)')

There is an upward trend year over year: 2014 on average is better than 2013, and 2015 better than 2014. And at the end of each year there is an obvious spike of sales.

In [ ]:
# let's look at trend of revenue
month_revenue = monthly_train_lk.groupby(['date_block_num','year','month'],as_index=False)['total_revenue'].sum()

plt.figure(figsize=(12,5))
sns.lineplot(x='date_block_num',y='total_revenue',data=month_revenue).set_title('Trend of Monthly Revenue')

Revenue shows the similar trend as items sold.

In [ ]:
# let's then look at sales data at category level
category_sum = monthly_train_lk.groupby(['item_category_id'],sort=False)['total_item_cnt'].agg([('total_item_cnt','sum')]).reset_index().sort_values(by='total_item_cnt',ascending=False)
#category_sum = category_sum.sort_values(by='total_item_cnt',ascending=False)
print(category_sum.head())
plt.figure(figsize=(12,5))
sns.barplot(x='item_category_id',y='total_item_cnt',data=category_sum,palette='viridis').set_title('Monthly Sales by Categories')

In [ ]:
category_rev_sum = monthly_train_lk.groupby(['item_category_id'],sort=False)['total_revenue'].agg([('total_revenue','sum')]).reset_index().sort_values(by='total_revenue',ascending=False)
print(category_rev_sum.head())
plt.figure(figsize=(12,5))
sns.barplot(x='item_category_id',y='total_revenue',data=category_rev_sum,palette='viridis').set_title('Monthly Revenue by Categories')

Looks a few number of categories take up a large portion of the total sell count. Yet top revenue contributors are not necessarily the most popular categories.

In [ ]:
shop_mean = monthly_train_lk.groupby(['shop_id'],as_index=False)['total_item_cnt'].mean()
shop_sum = monthly_train_lk.groupby(['shop_id'],as_index=False)['total_item_cnt'].sum()
print(shop_sum.head())
plt.figure(figsize=(12,5))
sns.barplot(x='shop_id',y='total_item_cnt',data=shop_sum,palette='viridis').set_title('Monthly Sales by Shops')

In [ ]:
shop_rev_sum = monthly_train_lk.groupby(['shop_id'],as_index=False)['total_revenue'].sum()
print(shop_rev_sum.head())
plt.figure(figsize=(12,5))
sns.barplot(x='shop_id',y='total_revenue',data=shop_rev_sum,palette='viridis').set_title('Monthly Revenue by Shops')

Most shops have a similar sell count. And shops that sell the most number of items are also top revenue contributors.

In [ ]:
del sales_train
del items
del item_cat
del shops
del negative_cnt
del dup_shop
del items_40
del test_shop_ids
del test_item_ids
del train_shop_ids
del train_item_ids
del temp_df
del month_sum
del month_sum_pivot
del month_revenue
del category_sum
del category_rev_sum
del shop_mean
del shop_sum
del shop_rev_sum

import gc 
gc.collect();

# Modeling and prediction

**Define a performance evaluation visualization function**

In [ ]:
# define a performance evaluation function and visualize the performance of different models
# The closer the points are to the middle dashed line the better are the predictions.
from numpy.polynomial.polynomial import polyfit

def model_performance_sc_plot(predictions,labels,title):
    min_val = max(max(predictions),max(labels))
    max_val = min(min(predictions),min(labels))
    performance = pd.DataFrame({'Label':labels,'Prediction':predictions})
    #performance = performance.astype(dtype={'Label':float,'Prediction':float})
    
    b, m = polyfit(performance['Prediction'], performance['Label'], 1)
    # plot the data
    plt.figure(figsize=(8,6))
    plt.plot([min_val,max_val],[min_val,max_val],'m--')
    plt.plot(performance['Prediction'],performance['Label'],'.')
    plt.plot(performance['Prediction'],b+m*performance['Prediction'],'-')
    plt.title(title)

***Part I : ARIMA***

I start with ARIMA mmodels because it's simple to implement without any parameter tuning, and quick to run.

In [ ]:
arima_data_df = monthly_train_lk.groupby(['date_block_num']).agg({'total_item_cnt':'sum'})
arima_data = list(arima_data_df['total_item_cnt'].values)

**Dickey-Fuller test**

In [ ]:
# perform dickey-fuller test to assess whether data is stationary
from statsmodels.tsa.stattools import adfuller
def stationary_test(data):
    #print('Is the data stationary ?')
    dftest = adfuller(data, autolag='AIC')
    print('Test statistic = {:.3f}'.format(dftest[0]))
    print('P-value = {:.3f}'.format(dftest[1]))
    print('Number of Observations = {:.0f}'.format(dftest[3]))
    print('Critical values :')
    for k, v in dftest[4].items():
        print('\t{}: {} - The data is {} stationary with {}% confidence'.format(k,v,'not' if v<dftest[0] else '',100-int(k[:-1])))
# let's run stationary test on raw data, de-trended data, and de-seasonalized data
print('Is the raw data stationary ?')
stationary_test(arima_data)

**AR model**

In [ ]:
# AR model
from statsmodels.tsa.ar_model import AR
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

model = AR(arima_data)
ar_model = model.fit()
yhat = ar_model.predict(12,len(arima_data)+18)

dataList = list(arima_data)
yhatList = list(yhat)

plt.style.use('seaborn-poster')
plt.figure()
plt.plot(dataList,label='Original')
plt.plot(yhatList,ls='--',label='Predicted')
plt.legend(loc='best')
plt.title('AR model')

In [ ]:
from sklearn.metrics import mean_squared_error

ar_rmse = np.sqrt(mean_squared_error(dataList,yhatList[0:34]))
print('AR RMSE: %.1f' % ar_rmse)

In [ ]:
# Initial approximation of parameters using Autocorrelation and Partial Autocorrelation Plots

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

plt.figure(figsize=(12,10))
ax = plt.subplot(211)
plot_acf(arima_data,lags=12,ax=ax)
plot_pacf(arima_data,lags=12,ax=ax)

**ARIMA model**

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import itertools

import warnings
from statsmodels.tools.sm_exceptions import HessianInversionWarning
warnings.simplefilter('ignore', HessianInversionWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning) 

best_aic = np.inf
best_order = None
best_order_seasonal = None
best_model = None
p = d = q = range(3)
pdq = list(itertools.product(p,d,q))
#seasonal_pdq = [(x[0],x[1],x[2],3) for x in list(itertools.product(p,d,q))]
output = []
for param in pdq:
    try:
        model = ARIMA(arima_data,
                      order=param)
        results = model.fit()
        output.append('ARIMA{} - AIC:{}'.format(param,results.aic))
        temp_aic = results.aic
        if temp_aic<best_aic:
            best_aic = temp_aic
            best_order = param
            best_model = results
    except:
        continue
print('aic: {:6.5f} | order: {}'.format(best_aic,best_order))
#aic: 676.13694 | order: (1, 2, 2)

Based on the above technique to find lowest AIC, the optimal order would be (1,2,2). However when applying oder=(1,2,2) to the ARIMA model, it throws out 'SVD not converge' error. After trial and error, I found the next best thing: order=(2,0,1).

In [ ]:
# ARIMA model

model = ARIMA(arima_data,order=(2,0,1))
arima_model = model.fit(disp=False)
yhat = arima_model.predict(1,len(arima_data)+6)

dataList = list(arima_data)
yhatList = list(yhat)

plt.style.use('seaborn-poster')
plt.figure()
plt.plot(dataList,label='Original')
plt.plot(yhatList,ls='--',label='Predicted')
plt.legend(loc='best')
plt.title('ARIMA model')

In [ ]:
arima_rmse = np.sqrt(mean_squared_error(dataList,yhatList[0:34]))
print('ARIMA RMSE: %.1f' % arima_rmse)

We can see that although ARIMA model almost acurately capture the historical patterns of the data, but it didn't reflect the huge spike for the end of year data point. Let's see if SARIMAX model remedy this issue with additional parameter for seasonality.

**SARIMAX**

In [ ]:
# SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(arima_data,
                order=(2,0,1),
                seasonal_order=(2,0,1,12))
sarima_model = model.fit(disp=False)
yhat = sarima_model.predict(1,len(arima_data)+6)

dataList = list(arima_data)
yhatList = list(yhat)

plt.style.use('seaborn-poster')
plt.figure()
plt.plot(dataList,label='Original')
plt.plot(yhatList,ls='--',label='Predicted')
plt.legend(loc='best')
plt.title('SARIMAX model')

In [ ]:
sarima_rmse = np.sqrt(mean_squared_error(dataList,yhatList[0:34]))
print('SARIMAX RMSE: %.1f' % sarima_rmse)

The SARIMAX model captures the huge spike for the year-end data point, but at the expense of accuracy. 

Another apparent flaw of the above AR/MA models is that they couldn't deal with hirarchical data. And it's not efficient to run an AR/MA model going through every combination of item_id and shop_id. Next for fun, I will explore a new technique called FB prophet.

**FB Prophet**

In [ ]:
fb_data = monthly_train_lk.groupby(['date_block_num'])['total_item_cnt'].sum()
fb_data.index = pd.date_range(start='2013-01-01',end='2015-10-01',freq='MS')
fb_data = fb_data.reset_index()
fb_data.head()

In [ ]:
from fbprophet import Prophet
fb_data.columns=['ds','y']
model = Prophet(yearly_seasonality=True)
model.fit(fb_data)

In [ ]:
fb_data_ext = model.make_future_dataframe(periods=6,freq='MS')
forecast = model.predict(fb_data_ext)
forecast[['ds','yhat','yhat_lower','yhat_upper']]

In [ ]:
dataList = arima_data

In [ ]:
plt.style.use('seaborn-poster')
plt.figure()
plt.plot(dataList,label='Original')
plt.plot(forecast['yhat'],ls='--',label='Predicted')
plt.legend(loc='best')
plt.title('FB Prophet model')

In [ ]:
fb_rmse = np.sqrt(mean_squared_error(dataList,forecast['yhat'][0:34]))
print('FB Prophet RMSE: %.1f' % fb_rmse)

The FB prophet captures both the upward trend and the spike by the end of each year. But I didn't explore how to deal with hierarchical data thru this approach. And the RMSE value is still relatively large as I didn't tune up the parameters of the FB model. For now I will leave it here, and move on to learn about other models that can deal with hierarchical data. 

In [ ]:
del arima_data_df
del arima_data
del fb_data

gc.collect();

***Part II : LSTM***

This part I will explore the single layer LSTM model. Why LSTM? It doesn't need level shifts and can handle non-linear relation with massive (and potentially multivariate) data.

In [ ]:
lstm_data = monthly_train_lk[['date_block_num','shop_id','item_id','total_item_cnt','item_cnt_next_month']]
lstm_data_series = lstm_data.pivot_table(index=['shop_id','item_id'],columns='date_block_num',values='total_item_cnt',fill_value=0).reset_index()
lstm_data_series.head()

In [ ]:
lstm_data_series = pd.merge(test_sub,lstm_data_series,on=['item_id','shop_id'],how='left')
lstm_data_series.fillna(0,inplace=True)
lstm_data_series.drop(['shop_id','item_id'],inplace=True,axis=1)
lstm_data_series.head()

**Prepare train/validation/test data**

In [ ]:
# prepare X's and y's
# for X we will keep all columns execpt the last one 
X = np.expand_dims(lstm_data_series.values[:,:-1],axis=2)
# the last column is our label, ie. y
y = lstm_data_series.values[:,-1:]

In [ ]:
# split train and validation among X's and y's
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=0)

print("Train set", X_train.shape)
print("Validation set", X_valid.shape)

In [ ]:
X_test = np.expand_dims(lstm_data_series.values[:,1:],axis=2)
print("Test set", X_test.shape)

**Train and predict with LSTM model**

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras import optimizers
from keras.utils import plot_model

series_size = X_train.shape[1] # 12
n_feat = X_train.shape[2] # 1

epochs = 10
batch = 64
lr = 0.0001

lstm_model = Sequential()
lstm_model.add(LSTM(units = 64,input_shape = (series_size,n_feat)))
lstm_model.add(Dropout(0.4))
lstm_model.add(Dense(1))
lstm_model.summary()

adam = optimizers.Adam(lr)
lstm_model.compile(loss='mse',optimizer=adam)
plot_model(lstm_model,show_shapes=True,to_file='regular_lstm.png')

In [ ]:
lstm_model.fit(X_train,y_train,batch_size=batch,epochs=epochs,validation_data=(X_valid,y_valid),verbose=2)

**Evaluate model performance**

In [ ]:
lstm_train_pred = lstm_model.predict(X_train)
lstm_val_pred = lstm_model.predict(X_valid)

print('Train RMSE:', np.sqrt(mean_squared_error(y_train, lstm_train_pred)))
print('Validation RMSE:', np.sqrt(mean_squared_error(y_valid, lstm_val_pred)))

In [ ]:
model_performance_sc_plot(lstm_val_pred.flatten(),y_valid.flatten(),'Validation')

In [ ]:
lstm_pred = lstm_model.predict(X_test)

In [ ]:
lstm_sub_df = pd.DataFrame(test_sub.index.values,columns=['ID'])
lstm_sub_df['item_cnt_next_month'] = lstm_pred.clip(0.,20.)
lstm_sub_df.to_csv('lstm_submission.csv',index=False)
lstm_sub_df.head(10)

The LSTM model prediction result looks good. Although I only implemented the basic LSTM model without further tuning the model hyper parameters. And the time series data can be truncated in better shape. For now I will leave it here and move on to ensembling method.

In [ ]:
del lstm_data_series
del lstm_data

gc.collect();

***Part III : Ensemble learning***

This part I will try to ensemble several machine learning models with feature engineering techniques. It might be complicated than the two approaches earlier, though it allows the flexibility to add more features that might be helpful for prediction.

**Step 1: Train/validation/test split**

Previously we added lag6 and lag12 feature to the dataframe. After the first run of the selected models, it turns out that lag6 and lag12 features are not showing significant importance in the prediction. So next we will ignore lag6/lag12 features and re-split train/validation.

In [ ]:
# there are 34 blocks from train and validation sets, we will ignore the first 3 blocks (0~2) as the first 3 blocks are used to generate windowned features
# then among the rest of the blocks available, we will split as following:
# train size = (3~27)
# validation size = (28-32) 
# test size = (33)
monthly_train_lk_new = monthly_train_lk.drop(['total_item_cnt_lag_6','total_item_cnt_lag_12'],axis=1)

train_set = monthly_train_lk_new[(monthly_train_lk['date_block_num']>=3) & (monthly_train_lk['date_block_num']<28)].copy()
validation_set = monthly_train_lk_new[(monthly_train_lk['date_block_num']>=28) & (monthly_train_lk['date_block_num']<33)].copy()
test_set = monthly_train_lk_new[monthly_train_lk['date_block_num']==33].copy()

In [ ]:
# clean the nan values in the prediction label
train_set.dropna(subset=['item_cnt_next_month'],inplace=True)
validation_set.dropna(subset=['item_cnt_next_month'],inplace=True)

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

print('Train set records:', train_set.shape[0])
print('Validation set records:', validation_set.shape[0])
print('Test set records:', test_set.shape[0])

print('Train set records: %s (%.f%% of complete data)' % (train_set.shape[0], ((train_set.shape[0]/monthly_train_lk_new.shape[0])*100)))
print('Validation set records: %s (%.f%% of complete data)' % (validation_set.shape[0], ((validation_set.shape[0]/monthly_train_lk_new.shape[0])*100)))

In [ ]:
del monthly_train_lk
del monthly_train_lk_new

gc.collect();

**Step 2 : Prepare train, validation and test set**

In [ ]:
# create train and validation set labels
# we will also drop column item_category_id as it's not available in the test_sub data
X_train = train_set.drop(['date_block_num','item_cnt_next_month','item_category_id'],axis=1)
y_train = train_set['item_cnt_next_month'].astype(int)
X_validation = validation_set.drop(['date_block_num','item_cnt_next_month','item_category_id'],axis=1)
y_validation = validation_set['item_cnt_next_month'].astype(int)
#X_test = test_set.drop(['item_cnt_next_month'],axis=1)

In [ ]:
# convert id and date related features to integer
int_feat = ['shop_id', 'item_id', 'year', 'month']

X_train[int_feat] = X_train[int_feat].astype(int)
X_validation[int_feat] = X_validation[int_feat].astype(int)

In [ ]:
combo = pd.concat([train_set,validation_set]).drop_duplicates(subset=['shop_id','item_id'],keep='last')
X_test = pd.merge(test_sub,combo,on=['shop_id','item_id'],how='left',suffixes=['','_'])
X_test['year'] = 2015
X_test['month'] = 10
X_test.drop('item_cnt_next_month',axis=1,inplace=True)
X_test[int_feat] = X_test[int_feat].astype(int)
X_test = X_test[X_train.columns]

In [ ]:
# check if there is null values in each X set
print(X_train.isnull().sum().sum())
print(X_validation.isnull().sum().sum())
print(X_test.isnull().sum().sum())

In [ ]:
# let's tackle the missing value issue in X_test
for item in X_test['item_id'].unique():
    for col in X_test.columns:
        item_median = X_test[(X_test['item_id']==item)][col].median()
        X_test.loc[(X_test[col].isnull()) & (X_test['item_id']==item),col] = item_median
        X_test.loc[np.isnan(X_test[col]) & (X_test['item_id']==item),col] = item_median
# if median isn't available for a certain item, calculate median based on items in the same shop
for shop in X_test['shop_id'].unique():
    for col in X_test.columns:
        shop_median = X_test[(X_test['shop_id']==shop)][col].median()
        X_test.loc[(X_test[col].isnull()) & (X_test['shop_id']==shop),col] = shop_median
        X_test.loc[np.isnan(X_test[col]) & (X_test['shop_id']==shop),col] = shop_median

#X_test.fillna(X_test.mean(), inplace=True)
X_test[int_feat] = X_test[int_feat].astype(int)

In [ ]:
# double check if there is missing value issue left
print(X_train.isnull().sum().sum())
print(X_validation.isnull().sum().sum())
print(X_test.isnull().sum().sum())
print(np.isnan(X_train).sum().sum())
print(np.isnan(X_validation).sum().sum())
print(np.isnan(X_test).sum().sum())

print(y_train.isnull().sum())
print(y_validation.isnull().sum())
print(np.isnan(y_train).sum())
print(np.isnan(y_validation).sum())

**Step 3 : run individual models**

***Tree based models - XGBoost***

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

xgb_model = XGBRegressor(max_depth=8,
                        n_estimators=1000,
                        min_child_weight=300,
                        colsample_bytree=0.8,
                        subsample=0.8,
                        eta=0.3,
                        seed=42)
xgb_model.fit(X_train,y_train,
             eval_metric='rmse',
             eval_set=[(X_train,y_train),(X_validation,y_validation)],
             verbose=True,
             early_stopping_rounds=10)

In [ ]:
plt.rcParams['figure.figsize'] = (12,10)
plot_importance(xgb_model)

In [ ]:
xgb_train_pred = xgb_model.predict(X_train)
xgb_val_pred = xgb_model.predict(X_validation)
xgb_test_pred = xgb_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

print('Train RMSE: ',np.sqrt(mean_squared_error(y_train,xgb_train_pred)))
print('Validation RMSE: ',np.sqrt(mean_squared_error(y_validation,xgb_val_pred)))

In [ ]:
model_performance_sc_plot(xgb_val_pred,y_validation,'Validation')

***Tree based models - random forest***

In [ ]:
# remove 0 importance features based on the first round model running
rf_feat = ['shop_id', 'item_id', 'total_item_price', 'avg_item_price', 'total_item_cnt', 'avg_item_cnt', 'total_txn_cnt', 'total_revenue', 'avg_revenue', 'month', 'year', 'max_unit_item_price', 'price_change-', 'total_item_cnt_lag_1', 'total_item_cnt_lag_2', 'total_item_cnt_lag_3', 'trend', 'total_item_cnt_rolling_min','total_item_cnt_rolling_max', 'total_item_cnt_rolling_avg']
rf_X_train = X_train[rf_feat]
rf_X_val = X_validation[rf_feat]
rf_X_test = X_test[rf_feat]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=50,
                                max_depth=7,
                                random_state=0,
                                n_jobs=-1)
rf_model.fit(rf_X_train,y_train)

In [ ]:
print('Model params:', rf_model.get_params())

In [ ]:
# get feature importance
feat_score = pd.DataFrame(list(zip(rf_X_train.dtypes.index,rf_model.feature_importances_)),columns=['Feature','Score'])
feat_score = feat_score.sort_values(by='Score',ascending=False,inplace=False,kind='quicksort',na_position='last')

plt.rcParams['figure.figsize'] = (12,5)
g = feat_score.plot('Feature','Score',kind='bar',color='c')
g.set_title('Random Forest Feature Importance')
g.set_xlabel('')

for patch,score in zip(g.patches,feat_score['Score'].round(3)):
    height = patch.get_height()
    g.text(patch.get_x()+patch.get_width()/2,
          height+0.01,
          score,
          ha='center')

In [ ]:
rf_train_pred = rf_model.predict(rf_X_train)
rf_val_pred = rf_model.predict(rf_X_val)
rf_test_pred = rf_model.predict(rf_X_test)

In [ ]:
print('Train RMSE: ',np.sqrt(mean_squared_error(y_train,rf_train_pred)))
print('Validation RMSE: ',np.sqrt(mean_squared_error(y_validation,rf_val_pred)))

In [ ]:
model_performance_sc_plot(rf_val_pred,y_validation,'Validation')

***Linear models - linear regression***

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

num_feat = ['total_item_price', 'avg_item_price', 'total_item_cnt', 'avg_item_cnt', 'total_txn_cnt', 'total_revenue', 'avg_revenue','unit_item_price', 'total_item_cnt_lag_1', 'total_item_cnt_lag_2', 'total_item_cnt_lag_3', 'trend','total_item_cnt_rolling_min', 'total_item_cnt_rolling_max', 'total_item_cnt_rolling_avg']
cat_feat = ['shop_id', 'item_id', 'month', 'year']
#X_train[int_feat] = X_train[int_feat].astype(int)

lr_scaler = MinMaxScaler()
lr_scaler.fit(X_train[num_feat])
lr_X_train = lr_scaler.transform(X_train[num_feat])
lr_X_val = lr_scaler.transform(X_validation[num_feat])
lr_X_test = lr_scaler.transform(X_test[num_feat])

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(lr_X_train,y_train)

In [ ]:
feat_score = pd.DataFrame(list(zip(X_train.dtypes.index,lr_model.coef_)),columns=['Feature','Score'])
feat_score = feat_score.sort_values(by='Score',ascending=False,inplace=False,kind='quicksort',na_position='last')

plt.rcParams['figure.figsize'] = (12,10)
g = feat_score.plot('Feature','Score',kind='bar',color='c')
g.set_title('Linear Regression Feature Importance')
g.set_xlabel('')

#for patch,score in zip(g.patches,feat_score['Score'].round(3)):
#    height = patch.get_height()
#    g.text(patch.get_x()+patch.get_width()/2,
#          height+5,
#          score,
#          ha='center')

In [ ]:
lr_train_pred = lr_model.predict(lr_X_train)
lr_val_pred = lr_model.predict(lr_X_val)
lr_test_pred = lr_model.predict(lr_X_test)

In [ ]:
print('Train RMSE: ',np.sqrt(mean_squared_error(y_train,lr_train_pred)))
print('Validation RMSE: ',np.sqrt(mean_squared_error(y_validation,lr_val_pred)))

In [ ]:
model_performance_sc_plot(lr_val_pred,y_validation,'Validation')

**Step 4: Model ensemble**

Combine predictions from all above models

In [ ]:
# train set
combo_train = pd.DataFrame({#'catboost':catboost_val_pred,
                             'XGB':xgb_val_pred,
                             'random_forest':rf_val_pred,
                             'linear_reg':lr_val_pred})
                             #'KNN':knn_val_pred})
                             #'label':y_validation.values})
combo_train.head(10)

In [ ]:
# test set
combo_test = pd.DataFrame({#'catboost':catboost_test_pred,
                             'XGB':xgb_test_pred,
                             'random_forest':rf_test_pred,
                             'linear_reg':lr_test_pred})
                             #'KNN':knn_test_pred})
combo_test.head(10)

In [ ]:
combo_model = LinearRegression(n_jobs=-1)
combo_model.fit(combo_train,y_validation)

In [ ]:
combo_pred = combo_model.predict(combo_train)
final_pred = combo_model.predict(combo_test)

In [ ]:
print('Train RMSE:', np.sqrt(mean_squared_error(combo_pred, y_validation)))

In [ ]:
model_performance_sc_plot(combo_pred,y_validation,'Validation')

The result of ensembling method looks better than LSTM model. It might be that LSTM model isn't tuned well enough, or it might be that the ensemble method captures as much data feature as possible in the prediction since it combines linear, xgboost and random forest models.

In [ ]:
submission_df = pd.DataFrame(test_sub.index.values,columns=['ID'])
submission_df['item_cnt_next_month'] = final_pred.clip(0.,20.)
submission_df.to_csv('ensemble_submission.csv',index=False)
submission_df.head(10)